## Nested Mapping en ElasticSearch
En este cuaderno se explora la relación **Nested** (o anidada) entre dos objetos en *ElasticSearch*

In [1]:
from elasticsearch import Elasticsearch

Definiendo variables globales e instanciando
elasticserach service

In [61]:
HOST = 'localhost:9200'

es = Elasticsearch(hosts=HOST)
es.indices.get("_all")

{'.kibana': {'aliases': {},
  'mappings': {'properties': {'config': {'properties': {'buildNum': {'type': 'long'}}},
    'type': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'updated_at': {'type': 'date'}}},
  'settings': {'index': {'creation_date': '1580765876896',
    'number_of_shards': '1',
    'number_of_replicas': '1',
    'uuid': '0L3hpd7EQOuxdoz8uoauRw',
    'version': {'created': '7050299'},
    'provided_name': '.kibana'}}}}

Crear un  index llamado **es_nested_workshop** con mapping tipo parent-child, donde:

**Objeto Principal**: Usuario
* user_name
* user_code
* user_time_stamp

**Objeto Anidado (Nested)**: Sensor
* Todas las caracteristicas de las medidas ingresadas


In [62]:
mapping = {
  "settings": {
    "number_of_replicas": 0,
    "number_of_shards": 1
  },
  "mappings": {
    "properties": {
      "user_name":{
        "type": "text"
      },
      "user_code":{
        "type": "keyword"
      },
      "sensor":{
        "type": "nested",
        "properties": {
          "sensor_code":{
            "type":"keyword"
          },
          "sensor_time_stamp": {
            "type":"date",
          },
          "Vehicle's speed (in m/s)": {"type": "float"},
          "Shift number (0 = intermediate position)": {"type": "float"},
          "Engine Load (% of max power)"format":"epoch_millis"": {"type":"float"},
          "Total Acceleration (m/s^2)": {"type": "float"},
          "Engine RPM": {"type": "float"}
        }
      }
    }
  }
}

SyntaxError: invalid syntax (<ipython-input-62-0e04a66652f0>, line 25)

In [63]:
index_name = "es_nested_workshop"
resp = es.indices.create(index=index_name, body=mapping)
resp

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'es_nested_workshop'}

Ingreso de usuarios con PUT. Este comando permite mantener las relaciones de **routing**

In [64]:
user_1= {
  "user_name":"Efra",
  "user_code": 123,
  "sensor":[
    {
      "sensor_code": "e-1",
      "sensor_time_stamp": 1000166400.5,
      "Vehicle's speed (in m/s)": 0.055034000000000007,
      "Shift number (0 = intermediate position)": 0.0,
      "Engine Load (% of max power)": -1.0,
      "Total Acceleration (m/s^2)": 0.020479,
      "Engine RPM": 608.63
    },
    {
      "sensor_code": "e-2",
      "sensor_time_stamp": 1000166400.51,
      "Vehicle's speed (in m/s)": 0.056112999999999996,
      "Shift number (0 = intermediate position)": 0.0,
      "Engine Load (% of max power)": -1.0,
      "Total Acceleration (m/s^2)": 0.007929899999999998,
      "Engine RPM": 623.47
    }]
}

resp = es.index(index=index_name, id=1, body=user_1)
resp

{'_index': 'es_nested_workshop',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [65]:
user_2={
  "user_name": "Lenno",
  "user_code": 456,
  "sensor":{
    "sensor_code": "l-1",
    "sensor_time_stamp": 1000166400.53,
    "Vehicle's speed (in m/s)": 0.060233,
    "Shift number (0 = intermediate position)": 0.0,
    "Engine Load (% of max power)": -1.0,
    "Total Acceleration (m/s^2)": -0.009987200000000002,
    "Engine RPM": 653.14
  }
}
resp = es.index(index=index_name, id=2, body=user_2)
resp

{'_index': 'es_nested_workshop',
 '_type': '_doc',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

Se puede incrementar los datos de cualquier usuario con la función **update**

In [66]:
new_data = {
  "script":{
    "source": "ctx._source.sensor.add(params.datum)",
    "lang": "painless",
    "params": {
      "datum":{
        "sensor_code":"e-3",
        "sensor_time_stamp":  122222.05,
         "Vehicle's speed (in m/s)": -6.6663,
         "Shift number (0 = intermediate position)": -6.60,
         "Engine Load (% of max power)": -6.6,
         "Total Acceleration (m/s^2)": -66666.6,
         "Engine RPM": 6600} 
    }
  }
}
resp = es.update(index=index_name, id=1, body=new_data)
resp

RequestError: RequestError(400, 'mapper_parsing_exception', "failed to parse field [sensor.sensor_time_stamp] of type [date] in document with id '1'. Preview of field's value: '1.0001664005E9'")

Obtener todos los hijos de un usuario por Código de Usuario

In [39]:
resp = es.get(index=index_name, id=1)
resp

{'_index': 'es_nested_workshop',
 '_type': '_doc',
 '_id': '1',
 '_version': 5,
 '_seq_no': 5,
 '_primary_term': 1,
 'found': True,
 '_source': {'user_name': 'Efra',
  'user_code': 123,
  'user_time_stamp': 1580488444.05,
  'sensor': [{'sensor_code': 1,
    'sensor_time_stamp': 1000166400.5,
    "Vehicle's speed (in m/s)": 0.055034000000000007,
    'Shift number (0 = intermediate position)': 0.0,
    'Engine Load (% of max power)': -1.0,
    'Total Acceleration (m/s^2)': 0.020479,
    'Engine RPM': 608.63},
   {'sensor_code': 2,
    'sensor_time_stamp': 1000166400.51,
    "Vehicle's speed (in m/s)": 0.056112999999999996,
    'Shift number (0 = intermediate position)': 0.0,
    'Engine Load (% of max power)': -1.0,
    'Total Acceleration (m/s^2)': 0.007929899999999998,
    'Engine RPM': 623.47},
   {"Vehicle's speed (in m/s)": -6.6663,
    'Shift number (0 = intermediate position)': -6.6,
    'sensor_time_stamp': 15888444.05,
    'Total Acceleration (m/s^2)': -66666.6,
    'sensor_code

In [41]:
new_data = {
  "script":{
    "source": "ctx._source.sensor.add(params.datum)",
    "lang": "painless",
    "params": {
      "datum":{
        "sensor_code":7,
        "sensor_time_stamp":  15888444.15,
         "Vehicle's speed (in m/s)": -26.6663,
         "Shift number (0 = intermediate position)": -26.60,
         "Engine Load (% of max power)": -26.6,
         "Total Acceleration (m/s^2)": -266666.6,
         "Engine RPM": 600} 
    }
  }
}
resp = es.update(index=index_name, id=2, body=new_data)

RequestError: RequestError(400, 'illegal_argument_exception', '[Lenin][127.0.0.1:9300][indices:data/write/update[s]]')